In [1]:
import json
from copy import deepcopy

In [2]:
class Number:
    
    def __init__(self, arg=None, parent=None):
        if arg is not None:
            self.parent = parent
            if type(arg) == int:
                self.isLiteral = True
                self.value = arg
            else:
                self.isLiteral = False
                self.left = Number(arg[0], parent=self)
                self.right = Number(arg[1], parent=self)
    
    def magnitude(self):
        return self.value if self.isLiteral else 3 * self.left.magnitude() + 2 * self.right.magnitude()
    
    def __add__(self, other):
        result = Number()
        result.isLiteral = False
        result.left = deepcopy(self)
        result.right = deepcopy(other)
        result.left.parent = result
        result.right.parent = result
        result.parent = None
        return result
    
    def depth(self):
        node = self
        i = 0
        while node.parent is not None:
            i += 1
            node = node.parent
        return i
    
    def dive(self, side):
        node = self
        while not node.isLiteral:
            node = side(node)
        return node
    
    def immediatelyTo(self, direction):
        funcs = [lambda x: x.left, lambda x: x.right]
        to, other = tuple(funcs if direction == 'left' else funcs[::-1])
        node = self
        while node.parent is not None:
            if other(node.parent) == node:
                return to(node.parent).dive(other)
            node = node.parent
        return None
    
    def canExplode(self):
        return self.depth() >= 4 and not self.isLiteral and self.left.isLiteral and self.right.isLiteral
    
    def explode(self):
        left = self.immediatelyTo('left')
        if left is not None:
            left.value += self.left.value
        right = self.immediatelyTo('right')
        if right is not None:
            right.value += self.right.value
        
        del self.left
        del self.right
        
        self.isLiteral = True
        self.value = 0
    
    def canSplit(self):
        return self.isLiteral and self.value >= 10
    
    def split(self):
        self.isLiteral = False
        self.left = Number(self.value // 2, parent=self)
        self.right = Number(self.value - self.left.value, parent=self)
        del self.value
    
    def reduce(self):
        def traverse(node, condition):
            if condition(node):
                return node
            elif node.isLiteral:
                return None
            else:
                leftRes = traverse(node.left, condition)
                if leftRes is not None:
                    return leftRes
                else:
                    return traverse(node.right, condition)
        
        while True:
            toExplode = traverse(self, lambda node: node.canExplode())
            toSplit = traverse(self, lambda node: node.canSplit())
            
            if toExplode is not None:
                toExplode.explode()
            elif toSplit is not None:
                toSplit.split()
            else:
                break
        return self
    
    def __repr__(self):
        if self.isLiteral:
            return f'{self.value}'
        else:
            return f'[{self.left},{self.right}]'

In [3]:
with open('input.txt', 'r') as f:
    numbers = [Number(json.loads(l.rstrip())) for l in f.readlines() if l.rstrip()]

In [4]:
result = numbers[0]
for number in numbers[1:]:
    result = (result + number).reduce()
print(f'Part 1: {result.magnitude()}')

Part 1: 3359


In [5]:
max_mag = 0
for i, n1 in enumerate(numbers):
    for j, n2 in enumerate(numbers):
        if i != j:
            max_mag = max(max_mag, (n1 + n2).reduce().magnitude(), (n2 + n1).reduce().magnitude())

print(f'Part 2: {max_mag}')

Part 2: 4616
